# PyCity Schools Analysis pandas-challenge

* Using Pandas and Jupyter Notebook, create a report that includes the following data. Your report must include a written description of at least two observable trends based on the data.
---

In [156]:
# Dependencies and Setup
import pandas as pd
import os
# Load Files from local directory, used Jupyter Notebook from VSCode which needed specifying the path 
school_data_path = os.path.abspath(os.getcwd() + "/Resources/schools_complete.csv") 
student_data_path = os.path.abspath(os.getcwd() + "/Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_path ,index_col = False)
student_data = pd.read_csv(student_data_path,index_col = False)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
# school_data_complete.head()  
school_data_complete['budget'] = school_data_complete['budget'].apply(lambda x: "${:,.2f}".format((x)))
school_data_complete

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,"$1,910,635.00"
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,"$1,910,635.00"
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,"$1,910,635.00"
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,"$1,910,635.00"
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,"$1,910,635.00"
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,"$1,043,130.00"
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,"$1,043,130.00"
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,"$1,043,130.00"
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,"$1,043,130.00"


## District Summary

In [157]:
# Display the DataFrame of District Summary #Perform the necessary calculations and then create a high-level snapshot of the district's key metrics in a DataFrame.

# Total number of unique schools
school_count = len(school_data_complete['school_name'].unique())

# Total students
student_count = school_data_complete["student_name"].count()

# Total budget Budget cannot be calculated from the merged data frame because the ammounts are duplicated and returns an incredibly large number that is unrealistic. 
total_budget = school_data['budget'].sum()

# Average math score
average_math_score = round(school_data_complete['math_score'].mean(),2)

# Average reading score
average_reading_score = round(school_data_complete['reading_score'].mean() ,2)

# % passing math (the percentage of students who passed math with scores greather than or equal to 70)
passing_math_percentage = round((school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]) / float(student_count) * 100,2)

#% passing reading (the percentage of students who passed reading with scores greather than or equal to 70)
passing_reading_percentage = round((school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]) / float(student_count) *100,2)

# % overall passing (the percentage of students who passed math AND reading)
overall_passing_math_reading_percentage = round(((((school_data_complete[(school_data_complete["math_score"] >= 70)]).student_name.count() / student_count) * 100)  + (((school_data_complete[(school_data_complete["reading_score"] >= 70)]).student_name.count() / student_count) * 100)) /2,2)
district_summary = pd.DataFrame([{
    "Total Schools":school_count,
    "Total Students":student_count,
    "Total Budget":total_budget,
    "Average Math Score":round(average_math_score, 2),
    "Average Reading Score":round(average_reading_score, 2),
    "% Passing Math":round(passing_math_percentage, 2),
    "% Passing Reading":round(passing_reading_percentage, 2),
    "% Overall Passing":round(overall_passing_math_reading_percentage,2),
}])
district_summary['Total Budget'] = district_summary['Total Budget'].apply(lambda x: "${:,.2f}".format((x)))
district_summary['% Passing Math'] = district_summary['% Passing Math'].apply(lambda x: "{:,.2%}".format(x/100))
district_summary['% Passing Reading'] = district_summary['% Passing Reading'].apply(lambda x: "{:,.2%}".format(x/100))
district_summary['% Overall Passing'] = district_summary['% Overall Passing'].apply(lambda x: "{:,.2%}".format(x/100))
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428.00",78.99,81.88,74.98%,85.81%,80.39%


## School Summary

In [158]:
# Use the code provided to select the school type
# schools_names = school_data.set_index(["school_name"])  #Name is implied, comes with summary. 
schools_type = school_data.set_index(["school_name"])["type"]
schools_total_students = school_data.set_index(["school_name"])["size"]
schools_total_budget = school_data.set_index(["school_name"])["budget"] 
schools_per_student_budget = schools_total_budget/schools_total_students
schools_average_math_score = round(school_data_complete.groupby(["school_name"])["math_score"].mean(),2)
schools_average_reading_score = round(school_data_complete.groupby(["school_name"])["reading_score"].mean(),2)
percent_students_pass_math =  round((school_data_complete[school_data_complete.math_score >=70].groupby(["school_name"]).count()["student_name"] / schools_total_students) * 100, 2)
percent_students_pass_reading = round((school_data_complete[school_data_complete.reading_score >=70].groupby(["school_name"]).count()["student_name"] / schools_total_students) * 100, 2)
percent_overall_passing = round(((school_data_complete[school_data_complete.math_score >=70].groupby(["school_name"]).count()["student_name"] / schools_total_students) * 100  + (school_data_complete[school_data_complete.reading_score >=70].groupby(["school_name"]).count()["student_name"] / schools_total_students) * 100)/2,2)
per_school_summary = pd.DataFrame({
    "School Type":schools_type,
    "Total Students":schools_total_students,
    "Total School Budget":schools_total_budget,
    "Per Student Budget":schools_per_student_budget,
    "Average Math Score":schools_average_math_score,
    "Average Reading Score":schools_average_reading_score,
    "% Passing Math":percent_students_pass_math,
    "% Passing Reading":percent_students_pass_reading,
    "% Overall Passing":percent_overall_passing,
})
per_school_summary2 = per_school_summary.copy() # copy dataframe to get around the type error while displaying % 
per_school_summary2['Total School Budget'] = per_school_summary2['Total School Budget'].apply(lambda x: "${:,.2f}".format(x))
per_school_summary2['Per Student Budget'] = per_school_summary2['Per Student Budget'].apply(lambda x: "${:,.2f}".format(x))
per_school_summary2['% Passing Math'] = per_school_summary2['% Passing Math'].apply(lambda x: "{:,.2%}".format(x/100))
per_school_summary2['% Passing Reading'] = per_school_summary2['% Passing Reading'].apply(lambda x: "{:,.2%}".format(x/100))
per_school_summary2['% Overall Passing'] = per_school_summary2['% Overall Passing'].apply(lambda x: "{:,.2%}".format(x/100))
per_school_summary2 

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.05,81.03,66.68%,81.93%,74.31%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,95.59%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,73.36%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,73.80%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,95.27%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.29,80.93,66.75%,80.86%,73.81%
Holden High School,Charter,427,"$248,087.00",$581.00,83.80,83.81,92.51%,96.25%,94.38%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,73.64%


## Highest-Performing Schools (by % Overall Passing)

In [159]:
# Highest-Performing Schools (by % Overall Passing)
# Sort the schools by % Overall Passing in descending order and display the top 5 rows.
bottom_schools = pd.DataFrame(per_school_summary2.sort_values(by="% Overall Passing", ascending=False).head())
bottom_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.06,83.98,94.13%,97.04%,95.59%
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.42,83.85,93.27%,97.31%,95.29%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.35,83.82,93.39%,97.14%,95.27%
Pena High School,Charter,962,"$585,858.00",$609.00,83.84,84.04,94.59%,95.95%,95.27%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.27,83.99,93.87%,96.54%,95.20%


## Lowest-Performing Schools (by % Overall Passing)

In [160]:
# Lowest-Performing Schools (by % Overall Passing)
# Sort the schools by % Overall Passing in ascending order and display the top 5 rows.

bottom_schools = pd.DataFrame(per_school_summary2.sort_values(by="% Overall Passing", ascending=True).head())
bottom_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.84,80.74,66.37%,80.22%,73.29%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.71,81.16,65.99%,80.74%,73.36%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.07,80.97,66.06%,81.22%,73.64%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.10,80.75,68.31%,79.30%,73.80%


## Math Scores by Grade

In [161]:
# Math Scores by Grade
# Perform the necessary calculations to create a DataFrame that lists the average math score for students of each grade level (9th, 10th, 11th, 12th) at each school.
ninth_grade_math_average = round((school_data_complete[school_data_complete.grade == "9th"]).groupby(["school_name"]).mean()["math_score"], 2)
tenth_grade_math_average = round((school_data_complete[school_data_complete.grade == "10th"]).groupby(["school_name"]).mean()["math_score"], 2)
eleventh_grade_math_average = round((school_data_complete[school_data_complete.grade == "11th"]).groupby(["school_name"]).mean()["math_score"], 2)
twelfth_grade_math_average = round((school_data_complete[school_data_complete.grade == "12th"]).groupby(["school_name"]).mean()["math_score"], 2)
math_scores_by_grade = pd.DataFrame({
    "Average Math 9th Grade":ninth_grade_math_average,
    "Average Math 10th Grade":tenth_grade_math_average,
    "Average Math 11th Grade":eleventh_grade_math_average,
    "Average Math 12th Grade":twelfth_grade_math_average
})
math_scores_by_grade 

,Average Math 9th Grade,Average Math 10th Grade,Average Math 11th Grade,Average Math 12th Grade
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Scores by Grade

In [162]:
# Reading Scores by Grade
# Create a DataFrame that lists the average reading score for students of each grade level (9th, 10th, 11th, 12th) at each school.
ninth_grade_reading_average = round((school_data_complete[school_data_complete.grade == "9th"]).groupby(["school_name"]).mean()["reading_score"], 2)
tenth_grade_reading_average = round((school_data_complete[school_data_complete.grade == "10th"]).groupby(["school_name"]).mean()["reading_score"], 2)
eleventh_grade_reading_average = round((school_data_complete[school_data_complete.grade == "11th"]).groupby(["school_name"]).mean()["reading_score"], 2)
twelfth_grade_reading_average = round((school_data_complete[school_data_complete.grade == "12th"]).groupby(["school_name"]).mean()["reading_score"], 2)
reading_scores_by_grade  = pd.DataFrame({
    "Average Reading 9th Grade":ninth_grade_reading_average,
    "Average Reading 10th Grade":tenth_grade_reading_average,
    "Average Reading 11th Grade":eleventh_grade_reading_average,
    "Average Reading 12th Grade":twelfth_grade_reading_average
})
reading_scores_by_grade 

,Average Reading 9th Grade,Average Reading 10th Grade,Average Reading 11th Grade,Average Reading 12th Grade
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

In [163]:
# Scores by School Spending
# Create a table that breaks down school performance based on average spending ranges (per student).  
# Use cut when you need to segment and sort data values into bins. 
# This function is also useful for going from a continuous variable to a categorical variable. 
# For example, cut could convert ages to groups of age ranges. 
# Supports binning into an equal number of bins, or a pre-specified array of bins.
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]
per_school_summary["Averge Spending Ranges (per student)"] = pd.cut(schools_per_student_budget, spending_bins, labels= labels)
# Use pd.cut to categorize spending based on the bins.
spending_math_scores = round(per_school_summary.groupby(["Averge Spending Ranges (per student)"]).mean()["Average Math Score"], 2)
spending_reading_scores = round(per_school_summary.groupby(["Averge Spending Ranges (per student)"]).mean()["Average Reading Score"], 2)
spending_passing_math = round(per_school_summary.groupby(["Averge Spending Ranges (per student)"]).mean()["% Passing Math"], 2)
spending_passing_reading = round(per_school_summary.groupby(["Averge Spending Ranges (per student)"]).mean()["% Passing Reading"], 2)
overall_passing_spending = round(per_school_summary.groupby(["Averge Spending Ranges (per student)"]).mean()["% Overall Passing"], 2)
spending_passing_math
#Use the scores above to create a DataFrame called spending_summary.
spending_summary = pd.DataFrame({
    "Average math score": spending_math_scores,
    "Average reading score": spending_reading_scores,
    "% Passing math": spending_passing_math.apply(lambda x: "{:,.2%}".format(x/100)),
    "% Passing reading": spending_passing_reading.apply(lambda x: "{:,.2%}".format(x/100)),
    "% Overall passing": overall_passing_spending.apply(lambda x: "{:,.2%}".format(x/100)),
})
spending_summary

,Average math score,Average reading score,% Passing math,% Passing reading,% Overall passing
Averge Spending Ranges (per student),,,,,
<$585,83.45,83.94,93.46%,96.61%,95.04%
$585-630,81.90,83.16,87.13%,92.72%,89.93%
$630-645,78.52,81.62,73.48%,84.39%,78.94%
$645-680,77.00,81.03,66.16%,81.13%,73.65%


##  Scores by School Size

In [164]:
# Scores by School Size
# Use the following code to bin the per_school_summary. 
size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

per_school_summary["School Size"] = pd.cut(schools_total_students, size_bins, labels= size_labels)
per_size_math_scores = round(per_school_summary.groupby(["School Size"]).mean()["Average Math Score"], 2)
per_size_reading_scores = round(per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"], 2)
per_size_passing_math = round(per_school_summary.groupby(["School Size"]).mean()["% Passing Math"], 2)
per_size_passing_reading = round(per_school_summary.groupby(["School Size"]).mean()["% Passing Reading"], 2)
per_size_passing_spending = round(per_school_summary.groupby(["School Size"]).mean()["% Overall Passing"], 2)

size_summary = pd.DataFrame({
    "Average math score": per_size_math_scores,
    "Average reading score": per_size_reading_scores,
    "% Passing math": per_size_passing_math.apply(lambda x: "{:,.2%}".format(x/100)),
    "% Passing reading": per_size_passing_reading.apply(lambda x: "{:,.2%}".format(x/100)),
    "% Overall passing": per_size_passing_spending.apply(lambda x: "{:,.2%}".format(x/100)),
})
size_summary


,Average math score,Average reading score,% Passing math,% Passing reading,% Overall passing
School Size,,,,,
Small (<1000),83.82,83.93,93.55%,96.10%,94.82%
Medium (1000-2000),83.37,83.87,93.60%,96.79%,95.20%
Large (2000-5000),77.74,81.34,69.96%,82.77%,76.36%


##  Scores by School Type

In [165]:
# Use the per_school_summary DataFrame from the previous step to create a new DataFrame called type_summary.
# This new DataFrame should show school performance based on the "School Type".

per_type_math_scores = round(per_school_summary.groupby(["School Type"]).mean()["Average Math Score"], 2)
per_type_reading_scores = round(per_school_summary.groupby(["School Type"]).mean()["Average Reading Score"], 2)
per_type_passing_math = round(per_school_summary.groupby(["School Type"]).mean()["% Passing Math"], 2)
per_type_passing_reading = round(per_school_summary.groupby(["School Type"]).mean()["% Passing Reading"], 2)
per_type_passing_spending = round(per_school_summary.groupby(["School Type"]).mean()["% Overall Passing"], 2)

type_summary = pd.DataFrame({
    "Average math score": per_type_math_scores,
    "Average reading score": per_type_reading_scores,
    "% Passing math": per_type_passing_math.apply(lambda x: "{:,.2%}".format(x/100)),
    "% Passing reading": per_type_passing_reading.apply(lambda x: "{:,.2%}".format(x/100)),
    "% Overall passing": per_type_passing_spending.apply(lambda x: "{:,.2%}".format(x/100)),
})
type_summary

,Average math score,Average reading score,% Passing math,% Passing reading,% Overall passing
School Type,,,,,
Charter,83.47,83.90,93.62%,96.59%,95.10%
District,76.96,80.97,66.55%,80.80%,73.67%
